# Zoningtaxlots QAQC

### Objectives:
+ Combine `qc_versioncomparison.csv` with `qc_versioncomparisonnownullcount.csv`; sort by field name. (Resulting report will show __if the value changed to a different value or to/from a null value__).
+ Add a __difference__ column to `qc_frequencychanges.csv`; sort by field name.
+ Add two fields to the BBL diff report
    + Flag indicating that __lot intersects with a rezoning done since the last version__
    + Flag indicating that __the area of the lot (taken from DTM) has changed by more than +/- 10% since the last version__
+ Rename fields in BBL diff report for the fields showing the new data, using similar naming convention as used for previous data set, e.g., ZD1NEW.

In [37]:
import geopandas as gpd
import pandas as pd
import cartoframes
import os
from cartoframes.auth import Credentials
from sqlalchemy import create_engine
from dotenv import load_dotenv, find_dotenv
from pathlib import Path
import time
from shapely.wkb import dumps, loads
from shapely.wkt import loads as wkt_loads 
pd.set_option('display.max_columns', 500)
print(time.strftime("%m/%d/%Y %H:%M:%S"))

09/13/2019 16:21:16


In [3]:
%load_ext dotenv
%dotenv ../.env
credentials = Credentials(username=os.getenv('CARTO_USERNAME'), 
                          key=os.getenv('CARTO_KEY'))
cc = cartoframes.CartoContext(creds=credentials)

In [4]:
conn = create_engine(os.getenv('BUILD_ENGINE').replace('localhost', os.getenv('IP')))

In [5]:
recipe_conn = create_engine(os.getenv('RECIPE_ENGINE'))

In [52]:
query = '''
select field, count as diff_count, 
percent as diff_percent,  
        newnullcount, oldnullcount, 
        countnew as total_count_new, 
        countold as total_count_old, 
        (countnew-countold) as total_count_diff 
from(
    SELECT * FROM
        ztl_qc_versioncomparisoncount a 
    JOIN 
        ztl_qc_versioncomparisonnownullcount b
    USING (field)) c
JOIN
frequencychanges d
USING (field)
ORDER BY field;
'''
df_versioncomparison = pd.read_sql(sql=query, con=conn)
df_versioncomparison

field  diff_count  diff_percent  newnullcount  \
0      commercialoverlay1         0.0           0.0             0   
1      commercialoverlay2         0.0           0.0             0   
2   limitedheightdistrict         0.0           0.0             0   
3        specialdistrict1         0.0           0.0             0   
4        specialdistrict2         0.0           0.0             0   
5        specialdistrict3         0.0           0.0             0   
6         zoningdistrict1         0.0           0.0             0   
7         zoningdistrict2         4.0           0.0             3   
8         zoningdistrict3         0.0           0.0             0   
9         zoningdistrict4         0.0           0.0             0   
10          zoningmapcode         0.0           0.0             0   
11        zoningmapnumber         0.0           0.0             0   

    oldnullcount  total_count_new  total_count_old  total_count_diff  
0              0            74971            74971                 0  
1              0              165              165                 0  
2              0             3037             3037                 0  
3              0           101903           101903                 0  
4              0               80               80                 0  
5              0                1                1                 0  
6              0           858394           858394                 0  
7              3            19856            19858                -2  
8              0              207              207                 0  
9              0               14               14                 0  
10             0            15713            15713                 0  
11             0           858552           858552                 0

In [54]:
query = '''
SELECT bblnew, bblprev, 
        zd1new, zd1prev, zd2new, zd2prev, zd3new, zd3prev, zd4new, zd4prev, 
        zmcnew, zmcprev, zmnnew, zmnprev, 
        co1new, co1prev, co2new, co2prev, 
        sd1new, sd1prev, sd2new, sd2prev, sd3new, sd3prev, 
        lhdnew, lhdprev, 
        inzonechange, mihflag, mihoption, 
        geom from bbldiffs;
'''
bbldiffs = gpd.GeoDataFrame.from_postgis(sql=query, con=conn)
bbldiffs

bblnew     bblprev zd1new zd1prev zd2new zd2prev zd3new zd3prev zd4new  \
0  2026050035  2026050035   M3-1    PARK   PARK    M3-1   None    None   None   
1  4103930056  4103930056     R2     R5B    R5B      R2   None    None   None   
2  1021750070  1021750070   C4-4     R7A    R7A    C4-4   None    None   None   
3  4017400047  4017400047    R5A     R5A     R5    None   None    None   None   
4  4031700021  4031700021    R4B     R4B   None    R7-1   None    None   None   
5  1021750074  1021750074   C4-4     R7A    R7A    C4-4   None    None   None   
6  1021740050  1021740050   C4-4     R7A    R7A    C4-4   None    None   None   
7  1004680035  1004680035    R7A     R7A   None     R8B   None    None   None   
8  4095560041  4095560041   R4-1    R4-1   None     R6A   None    None   None   

  zd4prev zmcnew zmcprev zmnnew zmnprev co1new co1prev co2new co2prev sd1new  \
0    None   None    None     6D      6D   None    None   None    None   None   
1    None   None    None    15B     15B   None    None   None    None   None   
2    None   None    None     3A      3A   None    None   None    None     IN   
3    None   None    None     9D      9D   None    None   None    None   None   
4    None   None    None    14A     14A   None    None   None    None   None   
5    None   None    None     3A      3A   C1-4    C1-4   None    None     IN   
6    None   None    None     3A      3A   None    None   None    None     IN   
7    None   None    None    12C     12C   C1-5    C1-5   None    None     TA   
8    None   None    None    18A     18A   C2-3    C2-3   None    None   None   

  sd1prev sd2new sd2prev sd3new sd3prev lhdnew lhdprev inzonechange mihflag  \
0    None   None    None   None    None   None    None         None    None   
1    None   None    None   None    None   None    None         None    None   
2      IN   None    None   None    None   None    None         None    None   
3    None   None    None   None    None   None    None         None    None   
4    None   None    None   None    None   None    None         None    None   
5      IN   None    None   None    None   None    None         None    None   
6      IN   None    None   None    None   None    None         None    None   
7      TA   None    None   None    None   None    None         None    None   
8    None   None    None   None    None   None    None         None    None   

  mihoption                                               geom  
0      None  (POLYGON ((-73.89841148214217 40.7964927255383...  
1      None  (POLYGON ((-73.76252130286763 40.6962809611559...  
2      None  (POLYGON ((-73.92714945492135 40.8637605880317...  
3      None  (POLYGON ((-73.86821214925357 40.7532895691665...  
4      None  (POLYGON ((-73.85381366411515 40.7261504254297...  
5      None  (POLYGON ((-73.92624460199279 40.8636300950106...  
6      None  (POLYGON ((-73.92545198321342 40.8624050354718...  
7      None  (POLYGON ((-73.98570154946874 40.7315930110750...  
8      None  (POLYGON ((-73.82514211779764 40.6864612769226...

In [61]:
cc.write(bbldiffs, 'ztl_qaqc_bbldiff')

Params: encode_geom, geom_col and everything in kwargs are deprecated and not being used any more


/opt/conda/lib/python3.7/site-packages/carto/sql.py:229: UserWarning: Batch SQL job created with job_id: 711086ae-6135-493f-8679-8de1d596e63b
  warnings.warn('Batch SQL job created with job_id: {job_id}'.format(job_id=data['job_id']))


Table successfully written to CARTO: https://dcpadmin.carto.com/dataset/ztl_qaqc_bbldiff


In [7]:
version_old = '2019/08/23'
version_new = '2019/09/03'

In [10]:
query1 = f'''
with dtm_compare as (
    SELECT bbl, geom_new, geom_old, (case when geom_new = geom_old then 0 else 1 end) flag 
    FROM 
    (SELECT bbl, ST_Multi(ST_Union(f.wkb_geometry)) as geom_new 
        FROM dof_dtm."{version_new}" f GROUP BY bbl ) a
    JOIN 
    (SELECT bbl, ST_Multi(ST_Union(f.wkb_geometry)) as geom_old 
        FROM dof_dtm."{version_old}" f GROUP BY bbl ) b
    USING(bbl))
, changed as (
    SELECT *, (st_area(geom_new)-st_area(geom_old))/st_area(geom_old) as area_diff 
    FROM dtm_compare
    WHERE flag = 1)
SELECT * FROM changed WHERE area_diff > 0.1 OR area_diff < -0.1;
'''

In [13]:
bbl_areachange = gpd.GeoDataFrame.from_postgis(sql=query, con=recipe_conn, geom_col='geom_new')

In [15]:
bbl_areachange_new = bbl_areachange[['bbl', 'geom_new', 'area_diff']]

In [39]:
bbl_areachange_old = bbl_areachange[['bbl', 'geom_old', 'area_diff']]
bbl_areachange_old.loc[:, 'geom_old'] = bbl_areachange_old.loc[:, 'geom_old'].apply(lambda x: wkt_loads(loads(x,  hex=True).wkt))
bbl_areachange_old=gpd.GeoDataFrame(bbl_areachange_old, geometry='geom_old')

In [125]:
from ipyleaflet import Map, basemaps, GeoData, basemap_to_tiles, LayersControl, FullScreenControl

m = Map(center=(40.730610, -73.935242), zoom=9)

dark_matter_layer = basemap_to_tiles(basemaps.CartoDB.DarkMatter)
m.add_layer(dark_matter_layer)

In [126]:
new = GeoData(geo_dataframe = bbl_areachange_new, 
              style={'color': 'green', 'opacity':10, 'weight':1.9, 'dashArray':'2', 'fillOpacity':0.6},
              name = 'new')

old = GeoData(geo_dataframe = bbl_areachange_old, 
              style={'color': 'red', 'opacity':10, 'weight':1.9, 'dashArray':'2', 'fillOpacity':0.6}, 
              name = 'old')

In [127]:
m.add_layer(new)
m.add_layer(old)
m.add_control(LayersControl())
m.add_control(FullScreenControl())

m

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …